In [1]:
import numpy as np
import importlib
import matplotlib as mpl
import matplotlib.pylab as plt
import time
from copy import deepcopy
import torch

from stg_energy.common import col, svg, samples_nd, get_labels_8pt
from stg_energy.fig5_cc.viz import vis_sample_plain
from pyloric.sbi_prior import create_prior, create_prior_general
from pyloric.sbi_wrapper import simulate, simulate_general, load_setup, get_time, stats
from stg_energy.fig7_temp.process_samples import merge_samples
import stg_energy.fig7_temp.viz

from stg_energy.fig2_histograms.energy import select_ss_close_to_obs
from parameters import ParameterSet

from joblib import Parallel, delayed

### Get posterior predictives at 27deg

In [2]:
setups_dict = ParameterSet('/home/michael/Documents/pyloric/pyloric/setups.prm')
hyperparams_11 = setups_dict['collect_samples_15deg_energy_ssRanges']

prior = create_prior(log=True)
general_prior = create_prior_general(hyperparams_11, log=True)

lims = torch.cat((prior.support.lower_bound.unsqueeze(0), prior.support.upper_bound.unsqueeze(0)), dim=0).T

params_mean_41 = general_prior.mean.numpy()
params_std_41 = general_prior.stddev.numpy()

In [3]:
hyperparams_11.model_params.temp = 283

for j in range(30, 500):
    data = np.load(f"../../results/temperature/sbi/27deg_uncond/posterior_samples/params_{j}.npz")
    sample_params_27 = data["conductance_params"]
    num_sims = 2520

    seeds = np.random.randint(100000, size=sample_params_27.shape[0])[None,].T.astype(np.float64)

    sample_params_27_with_seeds = np.concatenate((sample_params_27, seeds), axis = 1)
    print(sample_params_27_with_seeds.shape)

    def simulator(params_set):
        out_target = simulate_general(
            deepcopy(params_set[:-1].astype(np.float64)),
            hyperparams_11,
            seed=int(params_set[-1]),
        )
        return stats(out_target)

    simulation_outputs = Parallel(n_jobs=12)(
        delayed(simulator)(batch)
        for batch in sample_params_27_with_seeds[:num_sims]
    )

    np.savez(f"../../results/temperature/sbi/27deg_post_pred/201001_post_pred_at_11deg/201001_post_pred_with_41D_params_at_11deg_set_{j}.npz", 
             params=sample_params_27, stats=np.asarray(simulation_outputs), seeds=seeds.astype(int))

(2520, 42)


KeyboardInterrupt: 

### We might have more time, so draw samples from 27 deg posterior and simulate them at 11deg

In [3]:
flowfile = np.load("../../results/temperature/sbi/200511_flow27deg.npz", allow_pickle=True)
flow = flowfile['posterior'].tolist()

In [6]:
hyperparams_11.model_params.temp = 283

for j in range(500, 2500):
    samples = flow.sample(3100,) # sample a bit more than 2520 due to leakage
    sample_params_27 = samples[torch.exp(general_prior.log_prob(samples)) > 0.0][:2520].detach().numpy()
    
    print('j', j, samples.shape)
    
    num_sims = 2520

    seeds = np.random.randint(100000, size=sample_params_27.shape[0])[None,].T.astype(np.float64)

    sample_params_27_with_seeds = np.concatenate((sample_params_27, seeds), axis = 1)

    def simulator(params_set):
        out_target = simulate_general(
            deepcopy(params_set[:-1].astype(np.float64)),
            hyperparams_11,
            seed=int(params_set[-1]),
        )
        return stats(out_target)

    simulation_outputs = Parallel(n_jobs=12)(
        delayed(simulator)(batch)
        for batch in sample_params_27_with_seeds[:num_sims]
    )

    np.savez(f"../../results/temperature/sbi/27deg_post_pred/201001_post_pred_at_11deg/201001_post_pred_with_41D_params_at_11deg_set_{j}.npz", 
             params=sample_params_27, stats=np.asarray(simulation_outputs), seeds=seeds.astype(int))

j 500 torch.Size([3100, 41])


KeyboardInterrupt: 

### Reject if too far from obs

In [5]:
npz = np.load("../../results/experimental_data/summstats_prep845_082_0044.npz")
observation = npz["summ_stats"]

npz = np.load("../../results/experimental_data/trace_data_845_082_0044.npz")
t = npz["t"]

In [6]:
num_std = np.asarray(
    [0.02, 0.02, 0.02, 0.02, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
)

In [7]:
datafile = "../../results/prior_samples_after_classifier/samples_full_3.npz"
data = np.load(datafile)
stats_prior = data["stats"]
stats_mean = np.mean(stats_prior, axis=0)
stats_std = np.std(stats_prior, axis=0)

In [8]:
for j in range(30):
    data = np.load(f"../../results/temperature/sbi/27deg_post_pred/201001_post_pred_at_11deg/201001_post_pred_with_41D_params_at_11deg_set_{j}.npz")
    sample_params_27 = data["params"]
    sample_stats_27 = data["stats"]
    sample_seeds_27 = data["seeds"]
    
    good_params, good_dat, good_seeds = select_ss_close_to_obs(
        sample_params_27,
        sample_stats_27,
        sample_seeds_27,
        observation,
        num_std=num_std,
        stats_std=stats_std[:15],
    )
    data = np.savez(f"../../results/temperature/sbi/27deg_post_pred/201001_post_pred_at_11deg_close_to_obs/201001_post_pred_with_41D_params_at_11deg_close_to_obs_set_{j}.npz", 
                    params=good_params, stats=good_dat, seeds=good_seeds)

### Simulate at 27 degree

In [9]:
hyperparams_11.model_params.temp = 299

In [6]:
for j in range(30):
    print("j", j)
    data = np.load(f"../../results/temperature/sbi/27deg_post_pred/201001_post_pred_at_11deg_close_to_obs/201001_post_pred_with_41D_params_at_11deg_close_to_obs_set_{j}.npz")
    sample_params_27 = data["params"][0]
    sample_stats_at_11 = data["stats"][0]
    
    seeds = np.random.randint(100000, size=sample_params_27.shape[0])[None,].T.astype(np.float64)
    
    sample_params_27_with_seeds = np.concatenate((sample_params_27, seeds), axis=1)
    
    def simulator(params_set):
        out_target = simulate_general(
            deepcopy(params_set[:-1].astype(np.float64)),
            hyperparams_11,
            seed=int(params_set[-1]),
        )
        return stats(out_target)

    simulation_outputs = Parallel(n_jobs=12)(
        delayed(simulator)(batch)
        for batch in sample_params_27_with_seeds
    )

    simulation_outputs = np.asarray(simulation_outputs)
    
    backup_stats = deepcopy(simulation_outputs)
    condition = np.invert(np.any(np.isnan(backup_stats), axis=1))
    simulation_outputs = simulation_outputs[condition]
    sample_params_27 = sample_params_27[condition]
    sample_stats_at_11 = sample_stats_at_11[condition]
    seeds = seeds[condition]
    
    np.savez(f"../../results/temperature/sbi/27deg_post_pred/201001_post_pred_at_27_deg_close_to_obs_at_11_deg/201001_post_pred_at_27_deg_close_to_obs_at_11_deg_set_{j}.npz", 
             params=sample_params_27, stats=np.asarray(simulation_outputs), stats_at_11=sample_stats_at_11, seeds=seeds.astype(int))

j 0
j 1
j 2
j 3
j 4
j 5
j 6
j 7
j 8
j 9
j 10
j 11
j 12
j 13
j 14
j 15
j 16
j 17
j 18
j 19
j 20
j 21
j 22
j 23
j 24
j 25
j 26
j 27
j 28
j 29
